In [1]:
import os
from docx import Document
import pandas as pd
import re
import joblib

In [41]:
# تابع باز کردن فایل ها با کمی تغییرات
def open_file(file_path):
    file_name, file_ext = os.path.splitext(file_path)

    if file_ext == '.doc' or file_ext == '.docx':
        document = Document(file_path)
        full_text = "\n".join([paragraph.text for paragraph in document.paragraphs])
        return full_text
    elif file_ext == '.csv':
        df = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip',encoding="utf-8", quoting=1)
        return df
    elif file_ext == '.dat':
        df = pd.read_csv(file_path,delimiter='\t', header=None)
        return df
    elif file_path.endswith('.txt'):
        with open(file_path, 'r') as file:
            data = file.read()
            return data
    else:
        print(f"Unsupported file format for: {file_path}")
        return None

In [42]:
# این فانکشن برای تبدیل فاصله در افعال و جمع ها به نیم فاصله میباشد و برای ریشه یابی مناسب است
# از این فانکشن در تست آخر به دلیل ماهیت برنامه استفاده نشده
def convert_to_half_space(text):
  converted_text = re.sub(r'(?<=\S) (?=ها\b)', '‌', text)
  converted_text = re.sub(r'(?<=می) (?=\S)', '‌', converted_text)
  return converted_text

In [43]:
# تصحیح کلماتی نظیر عالللللیه که در آن نویسنده از شکل اشتباهی برای بیان احساسات استفاده کرده
def remove_consecutive_duplicates2(comment_list):
    modified_comments = []

    for text in comment_list:
        result = ""
        previous_char = None

        for char in text:
            if char != previous_char:
                result += char
            previous_char = char

        modified_comments.append(result)

    return modified_comments

In [44]:
# تغییر جمله ی محاوره ی به شکل درست فعل+است
def modify_sentences(sentences):
    modified_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if words[-1].endswith('ه'):
            words[-1] = words[-1][:-1]+ ' ' + 'است'
        modified_sentences.append(' '.join(words))
    return modified_sentences

# مثال
tokenized_sentences = ['این یک جمله خفنه', 'این یک جمله نیست.','این جمله ها قابل تغییره']
modified_sentencesss = modify_sentences(tokenized_sentences)
modified_sentencesss


['این یک جمله خفن است', 'این یک جمله نیست.', 'این جمله ها قابل تغییر است']

In [45]:
# تصحیح فاصله ها
def fix_half_space(text):
    half_space = "\u200C"
    normal_space = " "

    fixed_text = text.replace(half_space, normal_space)
    return fixed_text

استاپ ورد ها

In [46]:
stop_words=open_file(r'C:\Users\user\Downloads\stopwords.dat')

In [47]:
stop_words.tail(5)

,0
384,می‌یابد
385,لذا
386,زاده
387,گردد
388,اینجا


In [48]:
# تغییر لیست استاپ ورد ها در صورت نیاز زیرا در تست استفاده از تمامی کلات استاپ ورد تاثیر منفی در پردازش جمله داشت
#بسته به نیاز مسئله میتوان لیست اندیس هارا تغییر داد
indicies = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,23,31,33,46,49,68,72,77,90,102,109,388]
final_stop_words = stop_words.loc[indicies]
final_stop_words=final_stop_words.reset_index(drop=True)

In [49]:
def remove_stop_words(comment_column):
    stop_words = final_stop_words  
    words = comment_column.str.split()
    words = words.apply(lambda word_list: [word for word in word_list if word not in stop_words.iloc[:, 0].tolist()])
    return words.apply(lambda word_list: ' '.join(word_list))


In [50]:
df = open_file(r"C:\Users\user\Downloads\Snappfood - Sentiment Analysis.csv")

In [51]:
df

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0
...,...,...,...,...
69620,NaN,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1.0
69621,NaN,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1.0
69622,NaN,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0.0
69623,NaN,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1.0


In [52]:
df=df.drop('Unnamed: 0',axis=1)

In [53]:
df = df.dropna()

In [54]:
df['comment'] = modify_sentences(df['comment'])

In [55]:
df

,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0
...,...,...,...
69620,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1.0
69621,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1.0
69622,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0.0
69623,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1.0


In [56]:
df['comment'] = fix_half_space(df['comment'])

In [57]:
df

,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0
...,...,...,...
69620,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1.0
69621,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1.0
69622,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0.0
69623,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1.0


In [58]:
df['comment']=remove_stop_words(df['comment'])

In [59]:
df

,comment,label,label_id
0,واقعا حیف وقت بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر موقع ...,HAPPY,0.0
2,قیمت مدل اصلا کیفیتش سازگاری نداره، فقط ظاهر ف...,SAD,1.0
3,عالللی بود همه چه درست اندازه کیفیت خوب، امیدا...,HAPPY,0.0
4,شیرینی وانیلی فقط مدل بود.,HAPPY,0.0
...,...,...,...
69620,سلام فاکتور غذاهایی سفارش میدم احتیاج دارم موض...,SAD,1.0
69621,سایز پیتزا نسبت سفارشاتی قبلا گذشتم کم شده بود,SAD,1.0
69622,قارچ اضافه رو اضافه کرده بودم بودم اما اگر همی...,HAPPY,0.0
69623,همرو ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو هوا,SAD,1.0


In [60]:
df['comment']=remove_consecutive_duplicates2(df['comment'])

In [61]:
df

,comment,label,label_id
0,واقعا حیف وقت بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر موقع ...,HAPPY,0.0
2,قیمت مدل اصلا کیفیتش سازگاری نداره، فقط ظاهر ف...,SAD,1.0
3,عالی بود همه چه درست اندازه کیفیت خوب، امیداور...,HAPPY,0.0
4,شیرینی وانیلی فقط مدل بود.,HAPPY,0.0
...,...,...,...
69620,سلام فاکتور غذاهای سفارش میدم احتیاج دارم موضو...,SAD,1.0
69621,سایز پیتزا نسبت سفارشاتی قبلا گذشتم کم شده بود,SAD,1.0
69622,قارچ اضافه رو اضافه کرده بودم بودم اما اگر همی...,HAPPY,0.0
69623,همرو ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو هوا,SAD,1.0


In [62]:
from sklearn.model_selection import train_test_split

In [63]:
x_train,x_test,y_train,y_test = train_test_split(df['comment'],df['label_id'],test_size=0.2,random_state=17)

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 10,
                             max_df = 0.9999,
                             sublinear_tf = True,
                             use_idf = True,binary=True)
train_vectors = vectorizer.fit_transform(x_train)
test_vectors = vectorizer.transform(x_test)

In [65]:
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='rbf')
classifier_linear.fit(train_vectors, y_train)
prediction_linear = classifier_linear.predict(test_vectors)
# results
report = classification_report(y_test, prediction_linear, output_dict=True)
print('positive: ', report['0.0'])
print('negative: ', report['1.0'])

positive:  {'precision': 0.907732293697206, 'recall': 0.8078646812201822, 'f1-score': 0.8548917616461409, 'support': 6917.0}
negative:  {'precision': 0.8266370988781633, 'recall': 0.9177407675597393, 'f1-score': 0.8698098963695011, 'support': 6905.0}


In [ ]:
joblib.dump(vectorizer, 'vectorizer.joblib')
joblib.dump(classifier_linear, 'svm_model.joblib')